# 環境設定

In [1]:
import os
import sys

def _is_in_kaggle() -> bool:
  """Whether the current environment is in `Kaggle`."""
  return str(_dh[0]) == '/kaggle/working'


def _is_in_colab() -> bool:
  """Whether the current environment is in `Colab`."""
  return 'google.colab' in str(get_ipython())

In [2]:
if _is_in_kaggle():
  print('in kaggle')
elif _is_in_colab():
  print('in colab')
  from google.colab import drive
  drive.mount('/content/drive')  # mount google drive
  sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')  # path from drive
else:
  print('in local')

in colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tokenizer

tokenizerとは、文字列を機械学習モデルへ入力できる形式へ変換する為の機能を指し、行われる手順としては以下のようになる。

1. 文字列を分割する
2. (分割後の)文字列をID列に変換する

このとき1, 2はモデルの要求する形式ごとに分割サイズやID列は異なる。
特殊なファインチューニングを行うケースを除き、事前学習時に使ったトークナイザを使用することが推奨される。

まずトークナイザをそのまま利用したときの挙動を確認する。

In [ ]:
from transformers import AutoTokenizer, BertTokenizer

In [ ]:
sentences = ['Saito was so beside himself that he couldn’t tell fact from fiction.', 'Oda has a good sense of taste.']

In [ ]:
# AutoTokenizerでもBertTokenizerでも同じ
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# dictを見やすくしてくれる
import pprint

In [ ]:
pprint.pprint(tokenizer(sentences), width=200, compact=True)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 25185, 2430, 1108, 1177, 3148, 1471, 1115, 1119, 1577, 787, 189, 1587, 1864, 1121, 4211, 119, 102], [101, 152, 1810, 1144, 170, 1363, 2305, 1104, 5080, 119, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [ ]:
pprint.pprint(bert_tokenizer(sentences), width=200, compact=True)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 25185, 2430, 1108, 1177, 3148, 1471, 1115, 1119, 1577, 787, 189, 1587, 1864, 1121, 4211, 119, 102], [101, 152, 1810, 1144, 170, 1363, 2305, 1104, 5080, 119, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


このように、AutoTokenizerとBertTokenizerは同じトークナイザを使用することができるため、Bertを使う事を明示的にする意図がない場合はAutoTokenizerを使えば良い。

トークナイザをそのまま利用したときの出力は、input_idsとtoken_type_ids、attention_maskを返す。

また、モデルへの入力として利用する際は、paddingやreturn_tensorsなどの引数を指定する必要がある。

In [ ]:
pprint.pprint(tokenizer(
    sentences,
    add_special_tokens=True,
    padding=True,
    truncation=True,
    return_tensors='pt')
, width=200, compact=True)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[  101, 25185,  2430,  1108,  1177,  3148,  1471,  1115,  1119,  1577,
           787,   189,  1587,  1864,  1121,  4211,   119,   102],
        [  101,   152,  1810,  1144,   170,  1363,  2305,  1104,  5080,   119,
           102,     0,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


## トークナイザの保存
使用したトークナイザを保存したい場合はsave_pretrained()を利用する。

In [ ]:
tokenizer.save_pretrained('./my_tokenizer')

('./my_tokenizer/tokenizer_config.json',
 './my_tokenizer/special_tokens_map.json',
 './my_tokenizer/vocab.txt',
 './my_tokenizer/added_tokens.json',
 './my_tokenizer/tokenizer.json')

## エンコーディング
入力の文字列に対して、ID列(input_ids)のみを取得したい場合はencode()を利用する。

In [ ]:
input_ids = tokenizer.encode(sentences)
pprint.pprint(input_ids)

[101,
 25185,
 2430,
 1108,
 1177,
 3148,
 1471,
 1115,
 1119,
 1577,
 787,
 189,
 1587,
 1864,
 1121,
 4211,
 119,
 102,
 152,
 1810,
 1144,
 170,
 1363,
 2305,
 1104,
 5080,
 119,
 102]

一方で、トークン列を取得する際はtokenize()を利用する。

In [ ]:
tokens = tokenizer.tokenize(sentences)
tokens

['Sai',
 '##to',
 'was',
 'so',
 'beside',
 'himself',
 'that',
 'he',
 'couldn',
 '’',
 't',
 'tell',
 'fact',
 'from',
 'fiction',
 '.',
 'O',
 '##da',
 'has',
 'a',
 'good',
 'sense',
 'of',
 'taste',
 '.']

また、この時のtokensのリスト長をトークン数と呼ぶ

In [ ]:
token_length = len(tokens)
token_length

25

トークン列をID列に変換する際はconvert_tokens_to_ids()を利用する。

In [ ]:
ids_from_tokens = tokenizer.convert_tokens_to_ids(tokens)
ids_from_tokens

[25185,
 2430,
 1108,
 1177,
 3148,
 1471,
 1115,
 1119,
 1577,
 787,
 189,
 1587,
 1864,
 1121,
 4211,
 119,
 152,
 1810,
 1144,
 170,
 1363,
 2305,
 1104,
 5080,
 119]

encode()で得られるinput_idsとconvert_tokens_to_ids()で得られるids_from_tokensは特殊トークン（[CLS]、[SEP]など）を除き、同じID列を持つ。

## デコーディング
ID列を文字列に変換する(戻す)際はdecode()を利用する。

In [ ]:
tokenizer.decode(input_ids)

'[CLS] Saito was so beside himself that he couldn ’ t tell fact from fiction. [SEP] Oda has a good sense of taste. [SEP]'

In [ ]:
tokenizer.decode(ids_from_tokens)

'Saito was so beside himself that he couldn ’ t tell fact from fiction. Oda has a good sense of taste.'